## Usage

In [1]:
# import classes
from dataclasses import dataclass
from src import (
    MiddlewareBase,
    MiddlewareNextCallBase,
    MiddlewareableBase,
    RequestBase,
)


# define a payload
@dataclass
class Request(RequestBase):
    name: str


# create a middleware
class OneMiddleware(MiddlewareBase[Request]):
    async def handle(
        self,
        request: Request,
        next_call: MiddlewareNextCallBase[Request],
    ) -> None:
        print("OneMiddleware before")
        await next_call(request)
        print("OneMiddleware after")

        request.name = request.name + " from OneMiddleware"


# add the `middlewareable` mixin to the class that will use the middleware
class App(MiddlewareableBase[Request]):
    middlewares = [OneMiddleware]


# instantiate and use it
app = App()
request = Request(name="Hello")
await app.process_middlewares(request)

# check the result
print(request.name)

# output:
# OneMiddleware before
# OneMiddleware after
# Hello from OneMiddleware

OneMiddleware before
OneMiddleware after
Hello from OneMiddleware
